# AI Investigation: Using "Tennis serve game" to validate algorithm for finding Equilibrium of simple game theory games #
## AI Investigation Part 6 ##

#### *Goal: validate algorithm from Part 5 of using simulation to determine equilibrium of game theory games* ####
#### *further explores the new way to present the winloss function* ####

Creator: `Lang (Ron) Chen` 2022

In [1]:
import random
import numpy as np
import statistics as s
import scipy.stats
from collections import defaultdict

## Define Game

If both confesses, then each get 10 years in prison;

If A confesses and B does not, then A gets 1 year and B gets 25 years;

Vice versa if B confesses and A does not;

If both do not confess then they each get 3 years;

<img src = "./images/Tennis.png">

In [2]:
CHOICE = {'L': 0, 'C':1, 'R': 2}
OUTCOME1 = [[0.6, 0.6, 0.7], 
           [0.3, 0.1, 0.3],
           [0.9, 0.6, 0.2]]
OUTCOME2 = [[0.4, 0.7, 0.1],
           [0.4, 0.9, 0.4],
           [0.3, 0.7, 0.8]] # Be careful, this data must be entered vertically

*This is an experiment on upgrading the winloss function from using a host of if-else to a more lightweight structure. However this wouldn't work as well for games with more than two players, or if the CHOICEs are not discrete.*

*Also if two players have different payoffs then need two different OUTCOMEs, but this would still be more concise than writing two distinct winloss functions with massive if-else overheads*

In [3]:
def winloss1(player1, player2):
    return OUTCOME1[CHOICE[player1]][CHOICE[player2]]

In [4]:
def winloss2(player1, player2):
    return OUTCOME2[CHOICE[player2]][CHOICE[player1]]

In [5]:
def validation(choice, CHOICE):
    if "Inconclusive" in choice:
        return 'Inconclusive'
    if choice not in CHOICE:
        return False
    return True

**Simulation**

In [6]:
RUNS = 1000000

data1 = defaultdict(list)
data2 = defaultdict(list) # Using defaultdict makes the code more adaptable to different games - and the validation function would ensure that the final choice is valid. However it is up to the user to double check as we are not hardcoding the choices

sample = list()
victory = list()

choicekeys = list(CHOICE.keys())
for i in range(RUNS):
    obs1 = random.sample(choicekeys, 1)[0]
    obs2 = random.sample(choicekeys, 1)[0]
    
    data1[obs1].append(winloss1(obs1, obs2))
    data2[obs2].append(winloss2(obs1, obs2))
    
# This time we record the data for both players because we are interested in each's dominant strategy,  which will help us determine the equilibrium
# i.e. we are not as interested in, or equally interested in each's dominant strategy as well as the final equilibrium state

**Algorithm for final choice**

First: Manipulate data so that it is in a dictionary and the dictionary value is [mean, stdev, length]

In [7]:
stats1 = dict()
for choice in CHOICE:
    tmp = list()
    tmp.append(s.mean(data1[choice]))
    tmp.append(s.stdev(data1[choice]))
    tmp.append(len(data1[choice]))
    
    stats1[choice] = tmp
    
stats1

{'L': [0.6332809221167317, 0.04712195995856038, 332626],
 'C': [0.23305762569539204, 0.0943780699956027, 333445],
 'R': [0.5662488133705069, 0.2870476904930459, 333929]}

In [8]:
stats2 = dict()
for choice in CHOICE:
    tmp = list()
    tmp.append(s.mean(data2[choice]))
    tmp.append(s.stdev(data2[choice]))
    tmp.append(len(data2[choice]))
    
    stats2[choice] = tmp
    
stats2

{'L': [0.39963035357744586, 0.24491840171114135, 333562],
 'C': [0.5675021161384826, 0.23599632694410627, 333154],
 'R': [0.6006540968063273, 0.21593140763441968, 333284]}

In [9]:
def final_choice(stats):
    
    stats.sort(key = lambda x:x[1][0], reverse = True)

    # Then tests whether the second, third and so forth values contain 0 within their joint Confidence Intervals
    inconclusive_list = [stats[0][0]]
    inconclusive = False
    for i in range(1, len(stats)):
        if in_range(construct_CI(stats[0], stats[i])):
            inconclusive_list.append(stats[i][0])
            inconclusive = True
        else: # Because all values are sorted, if the current choice's joint Confidence Interval with the first ranked does not contain 0, then all the others wouldn't 
            break
            
    if inconclusive: # If inconclusive, return statement with the list of 'drawed' choices
        return f'Inconclusive: the following came to a draw {inconclusive_list}'
    
    return stats[0][0] # Else, return the dominant strategy

In [10]:
def construct_CI(stat1, stat2):
    """ Uses Welch's approximation to construct a joint CI of two means, unknown population variance and assuming different variance """
    
    xbar1 = stat1[1][0]
    xbar2 = stat2[1][0]
    s1 = stat1[1][1]
    s2 = stat2[1][1]
    n = stat1[1][2]
    m = stat2[1][2]
    
    r = (s1**2 /n + s2**2 /m)**2 /(s1**4 /(n**2 * (n-1)) + s2**4 /(m**2 * (m-1)))
    
    q = scipy.stats.t.ppf(0.95, df = r)
    
    poolsd = np.sqrt(s1**2 /n + s2**2 /m)
    
    CI = (xbar1 - xbar2 - q * poolsd, xbar1 - xbar2 + q * poolsd)
    
    print(f'{stat1[0]} {stat2[0]}: {CI}')
    print('\n')
    
    return CI

In [11]:
def in_range(CI):
    """ Helper function to check whether 0 is within the Confidence Interval """
    
    if CI[0] <= 0 and CI[1] >= 0:
        return True
    return False

In [12]:
stats1

{'L': [0.6332809221167317, 0.04712195995856038, 332626],
 'C': [0.23305762569539204, 0.0943780699956027, 333445],
 'R': [0.5662488133705069, 0.2870476904930459, 333929]}

In [13]:
result1 = final_choice(list(stats1.items()))
result1

L R: (0.06620406726443452, 0.06786015022801505)




'L'

In [14]:
stats2

{'L': [0.39963035357744586, 0.24491840171114135, 333562],
 'C': [0.5675021161384826, 0.23599632694410627, 333154],
 'R': [0.6006540968063273, 0.21593140763441968, 333284]}

In [15]:
result2 = final_choice(list(stats2.items()))
result2

R C: (0.03224049809162823, 0.034063463244061266)




'R'

**Validation**

In [16]:
validation(result1, CHOICE)

True

In [17]:
validation(result2, CHOICE)

True

**Emperical Testing**

In [19]:
victory1 = list()
victory2 = list()
for i in range(RUNS):
    victory1.append(winloss1(result1, result2))
    victory2.append(winloss2(result1, result2))

Equilibrium = (s.mean(victory1), s.mean(victory2))
Equilibrium

(0.7, 0.3)

This algorithm has not solved our problem because dominant strategy is L, C. 

The fundemental problem here is that we are still independently picking out the best choice based on us, without consideration for what choices the other player will make. Thus we may be picking a choice with a high mean score, but if the mean is dragged up by a cell which belongs to an opponent player's column with a low mean (they won't pick this strat as their dominant), then the 'real mean' won't be our best.

Need a backtracking and re-simulating algorithm to take into account of what the other player will pick